# PQC Scheduler Statistical Analysis

Statistical analysis of benchmark results for the PQC Scheduler.

Copyright (c) 2025 Dyber, Inc. All Rights Reserved.

In [ ]:
import json
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

## Load Benchmark Results

In [ ]:
def load_results(results_dir):
    """Load all benchmark results from a directory."""
    results = []
    for f in sorted(Path(results_dir).glob('run_*.json')):
        with open(f) as fp:
            data = json.load(fp)
            data['run_id'] = int(f.stem.split('_')[1])
            results.append(data)
    return pd.DataFrame(results)

# Load results - update path as needed
RESULTS_DIR = '../results/benchmark_latest'
df = load_results(RESULTS_DIR)
print(f"Loaded {len(df)} runs")
df.head()

## Descriptive Statistics

In [ ]:
key_metrics = [
    'throughput_mean',
    'latency_mean_us',
    'latency_p99_us',
    'sla_compliance',
    'security_score'
]

stats_df = df[key_metrics].describe()
stats_df.loc['cv'] = df[key_metrics].std() / df[key_metrics].mean()  # Coefficient of variation
stats_df

## Welch's t-test for Throughput Comparison

In [ ]:
def welch_ttest(group1, group2, alpha=0.05):
    """Perform Welch's t-test with effect size calculation."""
    t_stat, p_value = stats.ttest_ind(group1, group2, equal_var=False)
    
    # Cohen's d effect size
    pooled_std = np.sqrt((np.var(group1) + np.var(group2)) / 2)
    cohens_d = (np.mean(group1) - np.mean(group2)) / pooled_std
    
    # Degrees of freedom (Welch-Satterthwaite)
    n1, n2 = len(group1), len(group2)
    v1, v2 = np.var(group1, ddof=1), np.var(group2, ddof=1)
    df = ((v1/n1 + v2/n2)**2) / ((v1/n1)**2/(n1-1) + (v2/n2)**2/(n2-1))
    
    return {
        't_statistic': t_stat,
        'p_value': p_value,
        'degrees_of_freedom': df,
        'cohens_d': cohens_d,
        'significant': p_value < alpha
    }

# Example: Compare MDP-Optimal vs Static-PQC (simulated)
# In real analysis, load comparison data from different configurations
mdp_throughput = df['throughput_mean'].values
baseline_throughput = mdp_throughput * 0.807  # 23.7% improvement simulation

result = welch_ttest(mdp_throughput, baseline_throughput)
print("Welch's t-test Results:")
for k, v in result.items():
    print(f"  {k}: {v:.4f}" if isinstance(v, float) else f"  {k}: {v}")

## Bootstrap Confidence Intervals

In [ ]:
def bootstrap_ci(data, n_bootstrap=10000, ci=0.95, statistic=np.mean):
    """Compute bootstrap confidence interval using BCa method."""
    data = np.array(data)
    n = len(data)
    
    # Bootstrap samples
    boot_stats = np.array([
        statistic(np.random.choice(data, size=n, replace=True))
        for _ in range(n_bootstrap)
    ])
    
    # BCa correction
    theta_hat = statistic(data)
    
    # Bias correction
    z0 = stats.norm.ppf(np.mean(boot_stats < theta_hat))
    
    # Acceleration (jackknife)
    jack_stats = np.array([
        statistic(np.delete(data, i)) for i in range(n)
    ])
    jack_mean = np.mean(jack_stats)
    a = np.sum((jack_mean - jack_stats)**3) / (6 * np.sum((jack_mean - jack_stats)**2)**1.5)
    
    # Adjusted percentiles
    alpha = (1 - ci) / 2
    z_alpha = stats.norm.ppf(alpha)
    z_1_alpha = stats.norm.ppf(1 - alpha)
    
    alpha1 = stats.norm.cdf(z0 + (z0 + z_alpha) / (1 - a * (z0 + z_alpha)))
    alpha2 = stats.norm.cdf(z0 + (z0 + z_1_alpha) / (1 - a * (z0 + z_1_alpha)))
    
    ci_lower = np.percentile(boot_stats, alpha1 * 100)
    ci_upper = np.percentile(boot_stats, alpha2 * 100)
    
    return {
        'mean': theta_hat,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'ci_width': ci_upper - ci_lower
    }

# Compute CIs for key metrics
print("95% Bootstrap Confidence Intervals (BCa):")
print("=" * 60)
for metric in key_metrics:
    ci = bootstrap_ci(df[metric].values)
    print(f"{metric}:")
    print(f"  Mean: {ci['mean']:.4f}")
    print(f"  95% CI: [{ci['ci_lower']:.4f}, {ci['ci_upper']:.4f}]")
    print()

## Bonferroni Correction for Multiple Comparisons

In [ ]:
def bonferroni_correction(p_values, alpha=0.05):
    """Apply Bonferroni correction for multiple hypothesis testing."""
    n_tests = len(p_values)
    adjusted_alpha = alpha / n_tests
    
    results = []
    for name, p in p_values.items():
        results.append({
            'test': name,
            'p_value': p,
            'adjusted_alpha': adjusted_alpha,
            'significant': p < adjusted_alpha
        })
    return pd.DataFrame(results)

# Example with multiple comparisons
p_values = {
    'throughput': 1e-10,
    'latency': 2e-8,
    'sla_compliance': 5e-12,
    'security': 3e-6,
    'algorithm_switches': 1e-4,
    'fallback_events': 2e-3
}

corrected = bonferroni_correction(p_values, alpha=0.01)
print("Bonferroni Corrected Results (α=0.01):")
corrected

## Effect Size Interpretation

In [ ]:
def interpret_cohens_d(d):
    """Interpret Cohen's d effect size."""
    d = abs(d)
    if d < 0.2:
        return 'negligible'
    elif d < 0.5:
        return 'small'
    elif d < 0.8:
        return 'medium'
    else:
        return 'large'

# Effect sizes from paper
effect_sizes = {
    'MDP vs Static-PQC Throughput': 2.14,
    'Graceful Degradation Switch Time': 3.47,
    'Phased vs Big-Bang Migration Risk': 2.47
}

print("Effect Size Analysis:")
print("=" * 50)
for name, d in effect_sizes.items():
    print(f"{name}:")
    print(f"  Cohen's d: {d:.2f}")
    print(f"  Interpretation: {interpret_cohens_d(d)}")
    print()

## Summary Table for Paper

In [ ]:
def generate_summary_table(df):
    """Generate publication-ready summary table."""
    summary = []
    
    for metric in key_metrics:
        values = df[metric].values
        ci = bootstrap_ci(values)
        
        summary.append({
            'Metric': metric.replace('_', ' ').title(),
            'Mean ± Std': f"{np.mean(values):.2f} ± {np.std(values):.2f}",
            '95% CI': f"[{ci['ci_lower']:.2f}, {ci['ci_upper']:.2f}]",
            'Min': f"{np.min(values):.2f}",
            'Max': f"{np.max(values):.2f}"
        })
    
    return pd.DataFrame(summary)

summary_table = generate_summary_table(df)
print("Summary Table for Publication:")
summary_table

In [ ]:
# Export to LaTeX
latex_table = summary_table.to_latex(index=False, escape=False)
print("\nLaTeX Table:")
print(latex_table)